# Foundation Workshop Worksheet

# 1) Explore Unity Catalog and the Query Editor
-  On the Left-side menu, select "Catalog"
- Find the "`samples`" catalog and then click on the "`bakehouse`" schema; 
- There you will find the two tables we need for our analysis, amongst others: `sales_transactions` and `sales_customers`.
- Select one of those tables and you should see the overview of the table, which includes the column names, owner, last updated date, etc. You can click on the "**_Sample Data_**" tab to see a subset of the dataset you selected.
  - At the top of the results, you should see a search bar where you can query the data using natural language! Feel free to ask any question, even in German: "_**Wie vielle Kunden haben wir"**_ - How many customers do we have?
  ![](path)
- Click on the "_**Insights**_" tab to see how frequently this table has been used over the last 30 days. (This will probably be empty, but at the end of the workshop, there should be more interesting insights to look at.)

**Interesting info:** If you select the "Details" tab and look at the "Storage location", you can find the exact path where that table resides on your cloud provider. This is the Lakehouse architecture in action, where we seperate compute and storage.


- Now at the top right of the table, click on "**Create**" then "**Query**"
- This will open the Databricks SQL Editor where you can run the query (by clicking _**"Run all"**_)
- Try out some other queries in the SQL Editor and click "Run all"
  1) `SELECT * FROM samples.bakehouse.sales_transactions LIMIT 10;`
  2) `SELECT * FROM samples.bakehouse.sales_customers LIMIT 10;`

### Optional
- This is more advanced. Click on the "_**Show performance**_" button at the buttom of your results to see how your query performed. Learn more about the SQL query profile [here](https://docs.databricks.com/gcp/en/sql/user/queries/query-profile).






# 2) Explore Databricks Notebooks

To create a Notebook:
- On the left menu bar, click on the on "Workspace"
- Click on "Create folder" and call it "Databricks Foundation Worksheet"
- Within the folder, click on "Create" then select "Notebook"
- Rename the notebook to a name you like (add your name as a prefix so you can search for it quickly) by double clicking on the notebook tab at the top of the page. (It should currently have a name like "_Untitled Notebook ..._")
- To create a new **text cell, click on the "+ Text"** button below or above an active cell. Text cells allow you to write text, upload images, and much more. 
  - This cells are not executed as code and are used to provide clarity to the user.
- To create a new **code cell, click on the "+ Code"** button below or above an active cell
  - You can change the language of a code cell to Python, SQL, Scala, R or Markdown, depending on what you would like to us. 

Copy and paste the code snippets below into a cell in your Notebook.


In [0]:
-- View the rows in the sales_transaction table
SELECT * FROM samples.bakehouse.sales_transactions LIMIT 10;


In [0]:
-- View the rows in the sales_customers table
SELECT * FROM samples.bakehouse.sales_customers LIMIT 10;


### Data Exploration and Visaulisation

After running the code below, click on the "+" sign next to "Table" to add a visual
- For Viusalization type, choose "_Bar_"
- X column, choose "_day_"
- Y columns, choose "_total_transactions_"
- Then "Save"

To modify the visual, you can "Edit" it and explore other settings to customise your visual as you'd like.

In [0]:
-- Data Exploration & Visualisation
SELECT
  DATE_TRUNC('day', dateTime) AS day,
  COUNT(DISTINCT transactionID) AS total_transactions,
  COUNT(DISTINCT customerID) AS unique_customers,
  SUM(quantity) AS total_quantity_sold,
  SUM(totalPrice) AS total_revenue,
  AVG(totalPrice) AS avg_transaction_value
FROM samples.bakehouse.sales_transactions
GROUP BY DATE_TRUNC('day', dateTime)
ORDER BY day DESC
LIMIT 100

## Build an aggregate table

We will combine the customers and transaction tables to aggregate sales, customers and revenue data over city, country, product etc. This is just to demonstrate a typical data transformation/harmonisation step.

In [0]:
-- Combine the customers and transaction tables
SELECT 
  t.dateTime,
  c.city,
  c.state,
  c.country,
  t.product,
  t.paymentMethod,
  COUNT(DISTINCT t.transactionID) AS total_transactions,
  COUNT(DISTINCT t.customerID) AS unique_customers,
  SUM(t.quantity) AS total_quantity_sold,
  SUM(t.totalPrice) AS total_revenue,
  AVG(t.totalPrice) AS avg_transaction_value
FROM samples.bakehouse.sales_transactions t
LEFT JOIN samples.bakehouse.sales_customers c
  ON t.customerID = c.customerID
GROUP BY 
  t.dateTime,
  c.city,
  c.state,
  c.country,
  t.product,
  t.paymentMethod
LIMIT 100;


# 3) Automate this Notebook using "Lakeflow Jobs"

### Create the job
- On the Left, go to 'Jobs & Pipelines'
- Click on "Create" -> "Job"
- Rename the job to something like "YOURNAME-databricks-job"
- You can select "**Notebook**" or you could select "Add another task type" then select "**Notebook**"
- Take name: **Schedule Daily Report**
- Type: _[The option, "Notebook", should have been selected, if not you can select that]_
- Source: "**Workspace**"
- Path: _ Click on the text box, then navigate to the Notebook you worked on in the previous section, then click on **Confirm**_
- Compute: _Select the compute option you see under "Jobs Compute"_
- Leave everything else as default, then **Create task** (You can ignore the "parameters" in the screenshot)

![create](images/create_job.png)

### Add a schedule to your job
On the right of your job, under "Schedules & Triggers:
- Select **"Add trigger"**
- Trigger type: _Explore the "Scheduled" trigger type to see how frequently you could schedule this job._ Once a day, 5 times a week etc.
- _**Cancel this option** as we don't want to to create many scheduled jobs from everyone in this session._ 

### Run the job
Now we are ready to run our job. It will kick off the job on its own and you can monitor its progress.

- At the top right of the page, click on "Run now" to start this job. This will be a manual job run, since we triggered it ourselves. In production, you would set a scheduled trigger which starts off automatically.
- Click on the "Runs" tab to monitor your job run in realtime.
![Job Run](images/job_overview.png)

- Learn more about Lakeflow Jobs [here](https://docs.databricks.com/gcp/en/jobs/).

# 4) Create a dashboard with your data

Let us visualise the data we have prepared for our business executives:

![dashboard](images/dashboard.png)

## Add your dataset
- On the left side-bar menu, select "Dashboards"
- On the top-right, click "Create dashboard"
- Rename the dashboard by editing the "New Dashboard..." with any name you want by double clicking on it. Preferably add a prefix with your name, so you can find it easily.
- Rename the tab you are on - "Untitled page" to "Home" by double clicking on it.
- Click on the "Data" and under "Datasets", click on "Create from SQL"
- Copy the following code and paste it in the edit box (this is the same code from step 3)
  -  ``` SELECT 
  t.dateTime,
  c.city,
  c.state,
  c.country,
  t.product,
  t.paymentMethod,
  COUNT(DISTINCT t.transactionID) AS total_transactions,
  COUNT(DISTINCT t.customerID) AS unique_customers,
  SUM(t.quantity) AS total_quantity_sold,
  SUM(t.totalPrice) AS total_revenue,
  AVG(t.totalPrice) AS avg_transaction_value
FROM samples.bakehouse.sales_transactions t
LEFT JOIN samples.bakehouse.sales_customers c
  ON t.customerID = c.customerID
GROUP BY 
  t.dateTime,
  c.city,
  c.state,
  c.country,
  t.product,
  t.paymentMethod
LIMIT 100; ``` 

- Select the table you created earlier "bakehouse_agg". This is the dataset we will use to create the visualisations we need.

### Add your first Visual
- Click on the "Add visualisation icon". Resize it to match the image
- Under visualization, select the "Bar" chart
- For X axis, select "_**total_revenue**_"
- For Y axis, select "**_product_**"
  - Under the Y axis, click the 3 dots and under "By field" sort the Y axis
- For Color, click the "+" sign and select "_**total_revenue**_"
- Finally, click on the "Labels" toggle to show labels on each bar.

#### Take Home Task
- Follow the same steps to add other visuals.
- Modiy the theme to match your organisation's theme. ([Hint](https://docs.databricks.com/gcp/en/dashboards/settings#customize-colors))

# Add a filter
- Click on the filter icon which is between the "Data" and "Home" tab on the top left
- Click on the "+" icon to add a filter widget
- On the left, under "Filter" select "Range Slider"
- Under "Fields" add "_**total_quantity_sold**_"

Now you can filter your visualisations!

Find out more about designing and building your dashboards here: https://docs.databricks.com/gcp/en/dashboards/



# 5) Enable Genie so your users can talk to their data rigt from the dashboard

- At the top of the dashboard, towards the top right, click on the 3 dots, then click on "Settings"
- Select the "General" tab
- Select "Enable Genie"
- Select "Auto-generate Genie Space"
- Now, click "Publsih" on the top right of the dashboard
- Leave the default "Share data permission" then click "Publish".
- Click on the URL that appears; you should now see a "Ask Genie" button at the bottom of your dashboard.

- Ask the following questions and see what the responses are:
  - _what is the total revenue by day?_
  - _Which customer ordered the most in this period?_ 
  - _Show me total sales from Japan._

- Keep exploring Genie by selecting some of the suggested questions in the chat.

Find out more about setting up Genie Spaces here: https://docs.databricks.com/gcp/en/genie/set-up

# 6) Optional - Take Home
- Integrate Genie into Micfrosoft Teams: See [here](https://techcommunity.microsoft.com/blog/analyticsonazure/supercharge-data-intelligence-build-teams-app-with-azure-databricks-genie--azure/4442653) (If you would like to set this up, speak with your platform team)